In [1]:
%load_ext autoreload
%autoreload 2

from tqdm import tqdm
import pandas as pd
import numpy as np
import gzip
import json
import math
from datetime import datetime
from IPython.display import display
%matplotlib inline

In [2]:
from helper.manage_orderbooks_v2 import extract_orderbooks_for_one_currencypair, load_orderbook_snapshot
from helper.orderbook_container import OrderbookContainer
from helper.orderbook_trader import OrderbookTradingSimulator

In [3]:
filename = 'test'
data = load_orderbook_snapshot(filename, last_line=20)

100%|██████████| 20/20 [00:00<00:00, 20.13it/s]

Loaded 20 orderbooks from file 'test'.


In [4]:
orderbook = data[0]
print(orderbook)

orderbook2 = data[1]

diff = orderbook2.subtract(other=orderbook)  #.to_DataFrame(5)
#new = orderbook2.add(diff)

print(diff)

OrderbookContainer from 2016-11-08T10:00
  499 bids (best: 705.0)
  407 asks (best: 705.45)
  kind: 'orderbook'
OrderbookContainer from 2016-11-08T10:00
  23 bids (best: 673.62)
  27 asks (best: 705.45)
  kind: 'diff'


In [20]:
orderbook = data[0]
orderbook2 = data[1]

diff = orderbook.subtract(orderbook2)

display(diff.head())
print(diff)
ots = OrderbookTradingSimulator()
info = ots.check_current_value(orderbook, 4)
display(info)

,Amount,Type
678.31,-0.510201,bid
674.59,0.546871,bid
673.62,-0.545428,bid
705.45,-1.876972,ask
706.41,-0.045817,ask
706.81,-7.529719,ask


OrderbookContainer from 2016-11-08T10:01
  23 bids (best: 673.62)
  27 asks (best: 705.45)
  kind: 'diff'


{'cashflow': -2823.2013944685, 'trade_summary':           Amount
 705.45  2.772181
 706.17  0.052838
 706.19  0.645323
 706.89  0.105160
 707.18  0.424498, 'volume': 4.0}

In [6]:
orderbook.tail()

,Amount,Type
614.000000,1.838163,bid
614.320000,0.034763,bid
614.760000,0.000048,bid
705.224976,NaN,center
809.090000,0.200000,ask
810.000000,0.286798,ask
811.000000,0.000100,ask


In [7]:
ots = OrderbookTradingSimulator()
res = ots.trade_timespan(orderbooks=data[:2], volume=40, limit=705.5, timespan=2)
# res = ots.trade_timespan(orderbooks=data[2:4], volume=40, limit=705.5, timespan=2)

display(res.head(5))

,Amount
705.45,2.772181


,Amount
705.45,4.649153


/usr/lib64/python2.7/site-packages/pandas/core/indexing.py:140: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


,ASK,BID,CENTER,LIMIT,SPREAD,TIMESPAN,VOLUME,avg,cashflow,cost,cost_avg,high,low,volume_left,volume_traded
2016-11-08T10:00,705.45,705.0,705.224976,705.5,0.45,2,40,705.45,-5235.380014,1.669978,0.225024,705.45,705.45,32.578666,7.421334


  ##  adjust_orderbook  ##  


,Amount,Type
702.980000,0.913410,bid
703.000000,0.135358,bid
703.360000,1.700000,bid
703.700000,0.001250,bid
703.800000,5.276769,bid
704.624678,NaN,center
705.450000,4.649153,ask
706.170000,0.052838,ask
706.190000,0.645323,ask
706.410000,0.045817,ask


,Amount
705.45,-2.772181


,Amount
705.45,7.421334


  ##  adjust_orderbook END ##  


,Amount,Type
702.980000,0.913410,bid
703.000000,0.135358,bid
703.360000,1.700000,bid
703.700000,0.001250,bid
703.800000,5.276769,bid
704.624678,NaN,center
705.450000,4.649153,ask
706.170000,0.052838,ask
706.190000,0.645323,ask
706.410000,0.045817,ask
